In [140]:
import pandas as pd
df = pd.read_csv("./interfax.csv", parse_dates=True)
df['Text'] = df['Title'] + " " + df['Article'] + " " + df['Tags']
df = df.drop(columns=['Title', 'Article', 'Tags'])

In [141]:
df = df[df['Text'].notnull()]

In [142]:
df.head(10)
df.describe(include='all')

,Unnamed: 0,Date,Text
count,77953.000000,77041,77953
unique,NaN,74810,76008
top,NaN,2018-10-04 07:41:00,РЖД в ближайшие годы оборудует плацкартные ваг...
freq,NaN,6,6
mean,41047.695445,NaN,NaN
std,23765.369328,NaN,NaN
min,0.000000,NaN,NaN
25%,20380.000000,NaN,NaN
50%,41039.000000,NaN,NaN
75%,61749.000000,NaN,NaN


In [143]:
isnull = df['Date'].isnull()
for i in range(1, df.shape[0]):
    if isnull.iloc[i]:
        df['Date'].iloc[i] = df['Date'].iloc[i - 1]
df['Date'] = pd.to_datetime(df['Date'])

C:\Users\Egor Kolyuzhnov\AppData\Local\Temp\ipykernel_58308\3773003364.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Date'].iloc[i] = df['Date'].iloc[i - 1]


In [144]:
marks = ['!', '\"', "\'", '\(', '\)', '\-', '\+', ',', '\.', ';', ':', '<', '>', '\[', '\]', '%', '\$']
for m in marks:
    df['Text'] = df['Text'].replace(m, " ", regex=True)
numbers = [str(i) for i in range(10)]
for n in numbers:
    df['Text'] = df['Text'].replace(n, " ", regex=True)
df['Text'] = df['Text'].str.lower()

In [145]:
df = df.drop_duplicates(['Text'])

In [146]:
aux_words = [
    'даже',
    "еще",
    "именно",
    "как раз",
    "только",
    "просто",
    "прямо",
    "буквально",
    "хоть",
    "хотя бы",
    "почти",
    "чуть не",
    "чуть ли не",
    "едва ли",
    "все таки",
    "все же",
    "но",
    "да",
    "или",
    "то есть",
    "именно",
    "как то",
    "будь то",
    "то",
    "не",
    "ли",
    "как и",
    "чем",
    "тем",
    "словно",
    "точно",
    "в целом",
    "в общем",
    "в основном",
    "во многом",
    "во",
    "без малого",
    "в итоге",
    "в результате",
    "к слову",
    "в идеале",
    "в частности",
    "что",
    "а вдобавок",
    "а именно",
    "а также",
    "а то",
    "будто",
    "вдобавок",
    "даже",
    "же",
    "едва",
    "ежели",
    "если",
    "зато",
    "зачем",
    "или",
    "как",
    "когда",
    "коли",
    "из",
    "однако",
    "пока",
    "потому",
    "пусть",
    "также",
    "тоже",
    "хотя",
    "точно",
    "чем",
    "чтобы",
    "без",
    "близ",
    "во",
    "вместо",
    "вне",
    "для",
    "до",
    "за",
    "из",
    "из за",
    "из под",
    "кроме",
    "между",
    "на",
    "над",
    "об",
    "от",
    "перед",
    "по",
    "под",
    "при",
    "про",
    "ради",
    "со",
    "сквозь",
    "среди",
    "через",
    "здесь"
]
for w in aux_words:
    df['Text'] = df['Text'].replace(" " + w + " ", " ", regex=True)
    df['Text'] = df['Text'].replace("^" + w + " ", " ", regex=True)
    df['Text'] = df['Text'].replace(" " + w + "$", " ", regex=True)
letters = list(map(chr, range(ord('а'), ord('я')+1)))
for l in letters:
    df['Text'] = df['Text'].replace(" " + l + " ", " ", regex=True)
    df['Text'] = df['Text'].replace("^" + l + " ", " ", regex=True)
    df['Text'] = df['Text'].replace(" " + l + "$", " ", regex=True)

In [155]:
import re

class Porter:
    PERFECTIVEGROUND =  re.compile(u"((ив|ивши|ившись|ыв|ывши|ывшись)|((?<=[ая])(в|вши|вшись)))$")
    REFLEXIVE = re.compile(u"(с[яь])$")
    ADJECTIVE = re.compile(u"(ее|ие|ые|ое|ими|ыми|ей|ий|ый|ой|ем|им|ым|ом|его|ого|ему|ому|их|ых|ую|юю|ая|яя|ою|ею)$")
    PARTICIPLE = re.compile(u"((ивш|ывш|ующ)|((?<=[ая])(ем|нн|вш|ющ|щ)))$")
    VERB = re.compile(u"((ила|ыла|ена|ейте|уйте|ите|или|ыли|ей|уй|ил|ыл|им|ым|ен|ило|ыло|ено|ят|ует|уют|ит|ыт|ены|ить|ыть|ишь|ую|ю)|((?<=[ая])(ла|на|ете|йте|ли|й|л|ем|н|ло|но|ет|ют|ны|ть|ешь|нно)))$")
    NOUN = re.compile(u"(а|ев|ов|ие|ье|е|иями|ями|ами|еи|ии|и|ией|ей|ой|ий|й|иям|ям|ием|ем|ам|ом|о|у|ах|иях|ях|ы|ь|ию|ью|ю|ия|ья|я)$")
    RVRE = re.compile(u"^(.*?[аеиоуыэюя])(.*)$")
    DERIVATIONAL = re.compile(u".*[^аеиоуыэюя]+[аеиоуыэюя].*ость?$")
    DER = re.compile(u"ость?$")
    SUPERLATIVE = re.compile(u"(ейше|ейш)$")
    I = re.compile(u"и$")
    P = re.compile(u"ь$")
    NN = re.compile(u"нн$")

    def stem(word):
        word = word.lower()
        word = word.replace(u'ё', u'е')
        m = re.match(Porter.RVRE, word)
        if m is None:
            return word
        if m.groups():
            pre = m.group(1)
            rv = m.group(2)
            temp = Porter.PERFECTIVEGROUND.sub('', rv, 1)
            if temp == rv:
                rv = Porter.REFLEXIVE.sub('', rv, 1)
                temp = Porter.ADJECTIVE.sub('', rv, 1)
                if temp != rv:
                    rv = temp
                    rv = Porter.PARTICIPLE.sub('', rv, 1)
                else:
                    temp = Porter.VERB.sub('', rv, 1)
                    if temp == rv:
                        rv = Porter.NOUN.sub('', rv, 1)
                    else:
                        rv = temp
            else:
                rv = temp
            
            rv = Porter.I.sub('', rv, 1)

            if re.match(Porter.DERIVATIONAL, rv):
                rv = Porter.DER.sub('', rv, 1)

            temp = Porter.P.sub('', rv, 1)
            if temp == rv:
                rv = Porter.SUPERLATIVE.sub('', rv, 1)
                rv = Porter.NN.sub(u'н', rv, 1)
            else:
                rv = temp
            word = pre+rv
        return word
    stem=staticmethod(stem)

In [148]:
def stemming(s):
    s = s.split()
    s = map(lambda ss: Porter.stem(ss), s)
    return " ".join(s)
for i in range(df.shape[0]):
    df['Text'].iloc[i] = stemming(df['Text'].iloc[i])
df.head(10)

C:\Users\Egor Kolyuzhnov\AppData\Local\Temp\ipykernel_58308\630041118.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Text'].iloc[i] = stemming(df['Text'].iloc[i])


,Unnamed: 0,Date,Text
0,0,2023-04-09 12:16:00,марсел пят человек пострада обрушен жил дом мо...
1,1,2023-04-09 11:33:00,песк назва неадекватн реакц запад план рф пост...
2,2,2023-04-09 11:21:00,воен рф созда спецгрупп борьб западн танк моск...
3,3,2023-04-09 11:02:00,полиц ира будет отслежива ношен хиджаб помощ к...
4,4,2023-04-09 11:22:00,воен рф созда спецгрупп борьб западн танк моск...
5,5,2023-04-09 06:08:00,сша направ подводн лодк красн мор напряжен отн...
6,6,2023-04-09 08:33:00,случ эт ноч воскресен апрел обстрел израил сир...
7,7,2023-04-09 00:09:00,православн отмеча вербн воскресен москв апрел ...
8,8,2023-04-09 04:44:00,три ракет был выпущ сир сторон голанск высот а...
9,9,2023-04-08 21:47:00,динам спартак сыгра внич матч рпл собол заб св...


In [320]:
df['Text'].iloc[13]

'сотрудник tesla год пересыла друг друг виде камер наблюден чуж машин москв апрел interfax ru компан tesla заверя особен цен прав клиент частн жизн пишет the guardian оказа сотрудник компан гг пересыла друг друг запис видеокамер фот сдела камер электрокар отмеча некотор них могут счита довольн компрометир например один сотрудник призна пересыла виде котор владелец машин подход не полност гол работник дел запис авар например одн них tesla высок скорост сбива ребенк велосипед большинств фот виде довольн однообразн эт попа камер собак смешн дорожн знак эт кадр сотрудник tesla дела мем дел им коллег the guardian отмеча политик безопасн tesla указа запис камер анонимн никак могут быт соотнес личност владельц слов сотрудник программ показыва где был сдела зап дает возможн вычисл владельц эт конкретн электрокар сообща некотор запис был сдела машин стоя выключен парковк работник компан отмеча так образ могл видет например наход гараж водител tesla никак комментир дан сообщен tesla'

In [236]:
import datetime
words = [
    "газпром",
    "газ",
    "нефть",
    "бензин",
    "санкции",
    "кризис",
    "рубль",
    "рост",
    "падение",
    "цена",
    "подорожание",
    "удешевение"
]
_filter = df['Text'].str.contains(" " + stemming(words[0]) + " ")
for w in words:
    s = stemming(w)
    _filter |= df['Text'].str.contains(" " + s + " ")
    _filter |= df['Text'].str.contains("^" + s + " ")
    _filter |= df['Text'].str.contains(" " + s + "$")
filtered = df[_filter]

KeyboardInterrupt: 

In [243]:
before_start = filtered[filtered['Date'] >= datetime.datetime(2015, 1, 1)]
for i in range(before_start.shape[0]):
    date = before_start['Date'].iloc[i]
    before_start['Date'].iloc[i] = datetime.datetime(date.year, date.month, date.day)
before_start.describe(include="all", datetime_is_numeric=True)
before_start

C:\Users\Egor Kolyuzhnov\AppData\Local\Temp\ipykernel_58308\365184396.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  before_start['Date'].iloc[i] = datetime.datetime(date.year, date.month, date.day)


,Unnamed: 0,Date,Text
3,3,2023-04-09,полиц ира будет отслежива ношен хиджаб помощ к...
8,8,2023-04-09,три ракет был выпущ сир сторон голанск высот а...
10,10,2023-04-08,произошл ден суббот апрел ракет крым учен тайв...
13,13,2023-04-08,сотрудник tesla год пересыла друг друг виде ка...
14,14,2023-04-08,twitch wikimedia гроз миллион штраф неудален з...
...,...,...,...
81823,81823,2015-01-05,президент франц заяв возможн отм санкц отношен...
81824,81824,2015-01-05,правительств продолж субсидирова региональн ав...
81828,81828,2015-01-04,итог главн пользовател осваива интернет рост р...
81832,81832,2015-01-04,правительств росс обнов перечен жизнен важн ле...


In [323]:
funds = pd.read_excel("./gazprom-moscow-exchange.xlsx")
funds['Дата'] = pd.to_datetime(funds['Дата'], dayfirst=True)
funds['Change'] = 0
funds = funds.sort_values("Дата")
for i in range(1, funds.shape[0]):
    funds['Change'].iloc[i] = funds['Цена avg'].iloc[i] - funds['Цена avg'].iloc[i - 1]
funds.tail(10)

C:\Users\Egor Kolyuzhnov\AppData\Local\Temp\ipykernel_58308\4236339736.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  funds['Change'].iloc[i] = funds['Цена avg'].iloc[i] - funds['Цена avg'].iloc[i - 1]


,Дата,Bid,Ask,Цена open,Цена min,Цена max,Цена last,Цена avg,Оборот,Кол-во сделок,Change
9,2023-03-28,171.02,171.08,172.94,169.10,172.94,171.07,170.58,4614333299,74028,-1.24
8,2023-03-29,170.91,170.96,171.58,169.41,172.50,170.95,171.13,3841502396,59297,0.55
7,2023-03-30,171.71,171.72,170.78,170.02,171.80,171.71,171.13,2517969423,41954,0.00
6,2023-03-31,169.83,169.85,172.00,168.14,172.71,169.83,169.90,5331859695,89990,-1.23
5,2023-04-03,170.30,170.34,170.99,169.00,171.50,170.38,170.24,3879998447,68103,0.34
4,2023-04-04,172.54,172.55,171.00,170.14,176.00,172.55,173.81,12722343198,149732,3.57
3,2023-04-05,173.86,173.87,172.55,170.58,174.48,173.86,172.95,5691178694,68565,-0.86
2,2023-04-06,171.52,171.56,174.02,171.30,175.00,171.60,173.34,5244430462,78986,0.39
1,2023-04-07,173.07,173.09,171.50,171.29,173.18,173.09,172.30,3089494250,54104,-1.04
0,2023-04-10,174.54,174.55,173.55,173.23,175.20,174.54,174.18,4070633698,64768,1.88


In [324]:
changes = {}
for i in range(1, funds.shape[0]):
    start = funds["Дата"].iloc[i - 1]
    end = funds["Дата"].iloc[i]
    date = start
    while date < end:
        changes[str(date)] = funds['Change'].iloc[i - 1]
        date += datetime.timedelta(days=1)

In [325]:
before_start['Change'] = 0
for i in range(before_start.shape[0]):
    date = before_start["Date"].iloc[i]
    if str(date) in changes:
        change = changes[str(date)]
        before_start["Change"].iloc[i] = change
before_start.head(10)

C:\Users\Egor Kolyuzhnov\AppData\Local\Temp\ipykernel_58308\916556137.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  before_start['Change'] = 0
C:\Users\Egor Kolyuzhnov\AppData\Local\Temp\ipykernel_58308\916556137.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  before_start["Change"].iloc[i] = change


,Unnamed: 0,Date,Text,Change
3,3,2023-04-09,полиц ира будет отслежива ношен хиджаб помощ к...,-1.04
8,8,2023-04-09,три ракет был выпущ сир сторон голанск высот а...,-1.04
10,10,2023-04-08,произошл ден суббот апрел ракет крым учен тайв...,-1.04
13,13,2023-04-08,сотрудник tesla год пересыла друг друг виде ка...,-1.04
14,14,2023-04-08,twitch wikimedia гроз миллион штраф неудален з...,-1.04
15,15,2023-04-08,большинств россия увидел дефицит нужн иностра ...,-1.04
19,19,2023-04-08,лавр возглав делегац росс конференц стран сосе...,-1.04
22,22,2023-04-07,произошл ден пятниц апрел нов рекорд курс валю...,-1.04
23,23,2023-04-07,минторг сша рассматрива введен мер отношен про...,-1.04
24,24,2023-04-07,рубл заверш торг пятниц небольш повышен доллар...,-1.04


In [ ]:
for i in range(before_start.shape[0]):
    before_start['Change'].iloc[i] = "INCREASE" if before_start['Change'].iloc[i] > 0 else "DECREASE"
before_start  

C:\Users\Egor Kolyuzhnov\AppData\Local\Temp\ipykernel_58308\2587082583.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  before_start['Change'].iloc[i] = "INCREASE" if before_start['Change'].iloc[i] > 0 else "DECREASE"
